In [ ]:
!pip install -U transformers datasets scikit-learn accelerate


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
)

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

In [ ]:
train_path = "/Users/gwenythtan/Desktop/BT4012_FINALPROJ/train.csv"
df = pd.read_csv(train_path)

# Expecting columns: 'url' and 'target'
df = df.dropna(subset=["url", "target"])
df["target"] = df["target"].astype(int)

print("Train shape:", df.shape)
print(df.head())
print("Class balance:\n", df["target"].value_counts(normalize=True))


In [ ]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["target"],
    random_state=42,
)

print("Train size:", len(train_df))
print("Val size:", len(val_df))

In [ ]:
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))

keep_cols = ["url", "target"]
train_ds = train_ds.remove_columns(
    [c for c in train_ds.column_names if c not in keep_cols]
)
val_ds = val_ds.remove_columns(
    [c for c in val_ds.column_names if c not in keep_cols]
)

train_ds = train_ds.rename_column("target", "labels")
val_ds   = val_ds.rename_column("target", "labels")

In [ ]:
MODEL_NAME = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

MAX_LEN = 256
def tokenize_function(examples):
    return tokenizer(
        examples["url"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
    )

train_ds = train_ds.map(tokenize_function, batched=True)
val_ds   = val_ds.map(tokenize_function, batched=True)

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,
).to(device)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary", zero_division=0
    )

    try:
        auc = roc_auc_score(labels, logits[:, 1])
    except ValueError:
        auc = float("nan")

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": auc,
    }


In [ ]:
output_dir = "./deberta_checkpoints"

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_steps=50,
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

In [ ]:
metrics = trainer.evaluate()
print("Validation metrics:")
for k, v in metrics.items():
    print(f"{k}: {v}")

Performance Metrics

Validation metrics:
eval_loss: 0.1802661120891571
eval_accuracy: 0.9633679606342264
eval_precision: 0.9731843575418995
eval_recall: 0.9529540481400438
eval_f1: 0.9629629629629629
eval_roc_auc: 0.9939137401202904
eval_runtime: 34.0391
eval_samples_per_second: 53.732
eval_steps_per_second: 1.704
epoch: 2.0